In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


# Legg til src-mappen i sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))

from data_rens import DataRens
from statistiske_maal import Statistiske_maal
from data_manipulering import DataManipulering

# Temperatur:

* Oppretter et objekt av klassen DataRens og DataManipulering
* Bruker metodene for å opprette en dataframe og manipulere feil i denne

In [ ]:
# Oppretter filstien til JSONfil med nedbør
temp_fil = "../data/frost_temp.json"

# Oppretter et nedbørs-objekt i DataRens()
rens_temp = DataRens()
manip_temp = DataManipulering()


# Lager DataFrame av en JSON
df = rens_temp.fra_json_til_dataframe(temp_fil)


### Manipulerer dataen:
df_fjern_verdi = manip_temp.fjern_verdi_for_tilfeldig_aar(df)
df_legg_til_duplikater = manip_temp.legg_til_duplikater_for_tilfeldig_aar(df_fjern_verdi)


#Viser den manipulerte dataframen:
display(df_legg_til_duplikater.head())



# Datarens av temperatur
- Bruker metode fra DataRens for å rense dataframen

In [1]:
from data_rens import DataRens

# Opprett rens-objekt og sett inn DataFrame
rens = DataRens()
rens.sett_data(df_legg_til_duplikater)

# Vis widgetene
rens.vis_widget()


ModuleNotFoundError: No module named 'data_rens'

In [2]:
# Lager en kopi av df - for visualisering 
df_urenset = df_legg_til_duplikater.copy()

# Renser df
df_temp_renset = rens_temp.rens_DataFrame(df_legg_til_duplikater, vis_manglende=False)

# Plotter begge
rens_temp.plot_manglende_data(df_urenset, df_temp_renset)


print('Ferdig renset dataframe (de første fem radene):')
display(df_temp_renset.head())


NameError: name 'df_legg_til_duplikater' is not defined

# Beregning av statistiske mål:
*Beregninger*: <br>
Beregningene gjøres i 4 intervaller: <br>
1980-1989 <br>
1990-1999 <br>
2000-2009 <br>
2010-2020 <br>

Samt en beregning for hele intervallet i sin helhet (1980-2020)

**Beregning av gjennomsnitt**


In [ ]:
temp_mål = Statistiske_maal()

gjennomsnitt_temp = temp_mål.gjennomsnitt(df_temp_renset)

print("Gjennomsnitt i 4 intervaller + totalt: ")
for key, value in gjennomsnitt_temp.items():
    print(f"{key}: {value} °C")


**Beregning av median**

In [ ]:
median_temp = temp_mål.median(df_temp_renset)

print("Median i 4 intervaller + totalt: ")
for key, value in median_temp.items():
    print(f"{key}: {value} °C")

**Beregning av standardavvik**

In [ ]:
standardavvik_temp = temp_mål.standardavvik(df_temp_renset)

print("Standardavvik i 4 intervaller + totalt: ")
for key, value in standardavvik_temp.items():
    print(f"{key}: {value} °C")

# Visualisering

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))

from visualisering import vis_statistikk
from visualisering import visualiser_statistikk_per_tiår


In [ ]:
vis_statistikk(
    df_temp_renset["year"],
    df_temp_renset["value"],
    median=median_temp["Totalt"],
    gjennomsnitt=gjennomsnitt_temp["Totalt"],
    std=standardavvik_temp["Totalt"],
    tittel="Endring i temperatur over tid (1980–2020)",
    y_label="Grader (°C)"
)


In [ ]:
visualiser_statistikk_per_tiår(
    gjennomsnitt=gjennomsnitt_temp,
    median=median_temp,
    standardavvik=standardavvik_temp,
    tittel="Gjennomsnittlig årstemperatur per tiår (med avvik og median)",
    ylabel="Temperatur (°C)"
)


# Lagrer viktige variabler i med %store for å bruke videre

In [ ]:
# lager variablene for en samlet visualisering

%store df_temp_renset
%store gjennomsnitt_temp
%store standardavvik_temp
%store median_temp

# Prediktiv analyse

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))

from prediktiv_analyse import TrendModel

In [ ]:
temp_model = TrendModel(df_temp_renset)
temp_model.fit_models()
temp_model.plot_models()
temp_model.print_r2_scores()
temp_model.plot_prediction(2040, deg=1)
